<a href="https://colab.research.google.com/github/vadhri/distibuted-optimization/blob/main/graph-based-topology/SchocasticAdjacency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install igraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 32.1 MB/s eta 0:00:00


In [11]:
import numpy as np
import igraph as ig

N = 100
orig = np.random.uniform(500, 600, size=N)

g = ig.Graph.Full(n=N)

A = g.get_adjacency()
A = np.full((N, N), 1.0 / (N - 1))
np.fill_diagonal(A, 0)
A = np.linalg.matrix_power(A,10)

np.dot(A, orig)

print('is matrix power method close to sign consensus ? ', np.all(np.isclose(np.average(orig), np.dot(A, orig)[0], atol=1e-1)))


is matrix power method close to sign consensus ?  True


In [26]:
import numpy as np
import igraph as ig

def sinkhorn_knopp(A, max_iters=20000, tol=1e-8):
    """
    Make matrix doubly stochastic using Sinkhorn-Knopp algorithm.
    Assumes A is non-negative and has full support (i.e., connected graph).
    """
    A = A.copy().astype(float)
    for _ in range(max_iters):
        A /= A.sum(axis=1, keepdims=True)  # Row normalize
        A /= A.sum(axis=0, keepdims=True)  # Column normalize
        if np.allclose(A.sum(axis=1), 1, atol=tol) and np.allclose(A.sum(axis=0), 1, atol=tol):
            break
    return A

# 1. Create connected undirected graph
N = 40
g = ig.Graph.Erdos_Renyi(n=N, m=N*3, directed=False)
assert g.is_connected(), "Graph must be connected"

# 2. Get adjacency matrix
A = np.array(g.get_adjacency().data, dtype=float)

# 3. Make it doubly stochastic
W = sinkhorn_knopp(A)

# 4. Initialize state vector
x0 = np.random.uniform(500, 600, size=N)

# 5. Apply consensus iterations via matrix power (or repeated multiplication)
W_power = np.linalg.matrix_power(W, 100)
x_consensus = W_power @ x0

# 6. Report results
print("Original average:", np.mean(x0))
print("Consensus values (first 5):", x_consensus[:5])
print("All values ≈ average?", np.allclose(x_consensus, np.mean(x0), atol=1e-3))


Original average: 549.9770441044773
Consensus values (first 5): [549.19851899 549.19852215 549.19854864 549.19853585 549.19881427]
All values ≈ average? False
